# HealthCenta

## Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))
import importlib
import utils

importlib.reload(utils)


## Extract Data

In [ ]:
healthcenta_df= pd.read_excel("../data/raw/Sample data.xls")
healthcenta_df

## Explore Data

### Data Stats

In [ ]:
healthcenta_df.describe()

In [ ]:
healthcenta_df.info()

### Findings
There are 11 columns in the data.
#### Missing Data 
There are a number of columns with missing data: 
* `SP.1 , SP.2 , SP.3 , SP.4` have 1 missing data
* `SERVICE`: Contains more entries than other columns.
* `Unnamed: 6`: Entirely empty and should be removed from the dataset.

#### Data Cleaning Recommendations
- **Remove Unnecessary Columns**: The `Unnamed: 6` column should be removed as it contains no useful information.
- **Handle Missing Values**: Address missing values in the `SP.` columns. Options include:
  - Imputing missing values with an average or median value.
  - Removing rows with missing values if they are insignificant. This is the recomended option as the standard variation is high meaning data varies widely and imputing will not communicate the reality. Also the value is dependent on valious factors such as specialty, category and nature of procedure. 
- **Validate `SERVICE` Column**: Ensure that the entries in the `SERVICE` column align with expected values and clean any inconsistencies.

## Clean Data


### Make data lower case

In [ ]:
healthcenta_df = healthcenta_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


### Drop `Unamed: 6` column

In [ ]:
healthcenta_df=utils.drop_columns(healthcenta_df,"Unnamed: 6")
healthcenta_df

### Clean the Service column extract body part to its own column

In [ ]:
healthcenta_df=utils.categorize_body_parts_in_service_column(healthcenta_df)
healthcenta_df

### Drop rows with missing product index and rows with missing SP.1 to SP.4

In [ ]:
healthcenta_df=utils.drop_rows_by_column(healthcenta_df,"PRODUCT INDEX")
healthcenta_df=utils.drop_rows_by_column(healthcenta_df,"SP. 1")
healthcenta_df


## EDA

### Price Distribution:



In [61]:
price_columns = ['SP. 1', 'SP. 2', 'SP. 3', 'SP. 4']


In [ ]:

# Reshape data for better plotting
melted_data = healthcenta_df.melt(id_vars=['DEPARTMENT'], value_vars=price_columns, var_name='Service Provider', value_name='Price')

plt.figure(figsize=(12, 6))
sns.boxplot(x='DEPARTMENT', y='Price', hue='Service Provider', data=melted_data)
plt.xticks(rotation=90)
plt.title('Price Distribution by DEPARTMENT and Service Provider')
plt.xlabel('Department')
plt.ylabel('Price')
plt.show()
melted_data

### Service count by category

In [ ]:
category_counts = healthcenta_df['CATEGORY'].value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=category_counts.index, y=category_counts.values)
plt.title('Number of Services by Category')
plt.xlabel('Category')
plt.ylabel('Number of Services')
plt.xticks(rotation=45)
plt.show()


### Average Price by Service Provider

In [ ]:
avg_prices = healthcenta_df[price_columns].mean()

plt.figure(figsize=(10, 6))
sns.barplot(x=avg_prices.index, y=avg_prices.values)
plt.title('Average Price by Service Provider')
plt.xlabel('Service Provider')
plt.ylabel('Average Price')
plt.show()


### Average Prices by Department

In [ ]:
avg_prices_by_dept = healthcenta_df.groupby('DEPARTMENT')[price_columns].mean()

avg_prices_by_dept.plot(kind='bar', figsize=(12, 8))
plt.title('Average Prices by Department')
plt.xlabel('Department')
plt.ylabel('Average Price')
plt.xticks(rotation=45)
plt.show()


### Average Prices by Specialty

In [ ]:
specialty_prices = healthcenta_df.groupby('SPECIALTY')[price_columns].mean()

specialty_prices.plot(kind='bar', figsize=(12, 8))
plt.title('Average Prices by Specialty')
plt.xlabel('Specialty')
plt.ylabel('Average Price')
plt.xticks(rotation=45)
plt.show()


In [67]:
melted_data = healthcenta_df.melt(id_vars=['NATURE OF ENT. PROCEDURE', 'CATEGORY', 'SPECIALTY'], 
                        value_vars=price_columns, 
                        var_name='Service Provider', 
                        value_name='Price')





In [ ]:
# Calculate average price for each grouping
avg_price = melted_data.groupby(['NATURE OF ENT. PROCEDURE', 'CATEGORY', 'SPECIALTY'])['Price'].mean().reset_index()

# Pivot for easier plotting
pivot_table = avg_price.pivot_table(index=['NATURE OF ENT. PROCEDURE', 'CATEGORY'], 
                                    columns='SPECIALTY', 
                                    values='Price').reset_index()

pivot_table

In [ ]:
# Set plot style

# Plot for each category within each NATURE OF ENT. PROCEDURE
for (procedure, category_data) in pivot_table.groupby('NATURE OF ENT. PROCEDURE'):
    plt.figure(figsize=(12, 6))
    category_data.set_index('CATEGORY').drop(columns='NATURE OF ENT. PROCEDURE').plot(kind='bar', 
                                                                                      figsize=(14, 7),
                                                                                      rot=45,
                                                                                      title=f'Average Price by Specialty for {procedure}')
    plt.ylabel('Average Price')
    plt.xlabel('Category')
    plt.legend(title='Specialty')
    plt.tight_layout()
    plt.show()

#### Findings
Different Specialities can incur higher prices for example breast imaging by fluoroscopy speciality is more expensive than computed tomography scan.